In [2]:
def market_share_per_month(column, number_top_categories=-1,categories_to_combine=[]):
    """
    Computes the market share per month for the given product category column.
    
    Args:
    - column (str): Name of the column representing the product category.
    - number_top_categories (int, optional): Number of top categories to consider.
      Default is -1 which means all categories are considered.
    
    Returns:
    - category_performance_per_month (pd.DataFrame): Market share per month for each
      category in the column.
    """
    
    # Assert that column is a string
    assert isinstance(column, str), "column variable must be a string"
    
    # Assert that number_top_categories is an integer
    assert isinstance(number_top_categories, int), "top categories variable must be an integer"
    
    # Assert that categories_to_combine is a list
    assert isinstance(categories_to_combine, list), "categories too combine must be a list"

    # Load the transactions data
    column_names = ['t_dat', column]
    enhanced_transactions_path = "kaggle_data/enhanced_transactions.csv"
    enhanced_transactions = pd.read_csv(enhanced_transactions_path, usecols=column_names)
    
    # Drop missing dates and convert t_dat to datetime format
    enhanced_transactions = enhanced_transactions.dropna(subset=['t_dat'])
    enhanced_transactions['t_dat'] = pd.to_datetime(enhanced_transactions['t_dat'])
    
    
    # Compute the number of orders per month for each category
    orders_per_month = (
            enhanced_transactions
            .groupby([enhanced_transactions['t_dat'].dt.month, column]) 
            .agg({column:'count'})
            .rename(columns={column:'orders'})
            .reset_index()
    )
    
    
    
    if categories_to_combine:
        # Create a new dataframe with the combined categories

        orders_per_month.loc[orders_per_month[column].isin(categories_to_combine), column] = 'Other'
        orders_per_month = orders_per_month.groupby([column,'t_dat']).sum().reset_index()
        
        



    # Compute the top categories by order count
    if number_top_categories == -1:
        Top_categories = (
                orders_per_month
                .groupby([column]) 
                .agg({'orders':'sum'})
                .reset_index()
                .sort_values(by=['orders'],ascending=False)
                .reset_index(drop=True)
        )
    else: 
        Top_categories = (
                orders_per_month
                .groupby([column]) 
                .agg({'orders':'sum'})
                .reset_index()
                .sort_values(by=['orders'],ascending=False)
                .head(number_top_categories)
                .reset_index(drop=True)
        )
        

        
    # Compute the market share per month for each category
    market_share_per_month = (
        orders_per_month
        .assign(month_orders=orders_per_month.groupby('t_dat')['orders'].transform('sum'))
    )
    
    
    market_share_per_month = market_share_per_month.assign(percent_orders_per_month=market_share_per_month.orders / market_share_per_month.month_orders)
    market_share_per_month = market_share_per_month.pivot_table(values='percent_orders_per_month', index=[column], columns=['t_dat'], aggfunc=np.sum).fillna(0)
    
    # Compute the mean market share across all months for each category
    mean_market_share = market_share_per_month.mean(axis=1)
    

    
    # Compute the category seasonality by dividing the market share by the mean market share
    category_seasonality = market_share_per_month.div(mean_market_share, axis=0)
    
    # Merge with the top categories data
    category_performance_per_month = pd.merge(category_seasonality, Top_categories, on=column, how='inner')
    

    
    # Set the product category as the index
    category_performance_per_month = category_performance_per_month.set_index(column)
    
    category_performance_per_month = category_performance_per_month[category_seasonality.columns]

    
    
    return category_performance_per_month